# Apprentissage profond sur les jeux

### Exercice 1: Initialisaion de l'environement et des structures de données

In [2]:
import gymnasium as gym
import numpy as np

env = gym.make("Taxi-v3", render_mode = "human")

state_size = env.observation_space.n
action_size = env.action_space.n
print("Etats: ", state_size)
print("Actions: ", action_size)

policy_table = np.ones((state_size, action_size)) / action_size

value_table = np.zeros(state_size)
print("policy table:", policy_table)
print("value table: ", value_table)

Etats:  500
Actions:  6
policy table: [[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 ...
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]
value table:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0

### Exercice 2: Exploration et collecte d'episodes

In [4]:
env = gym.make("Taxi-v3", render_mode = "human")

n_episodes = 20

for episode in range(n_episodes):
    state = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        action = env.action_space.sample()

        next_state, reward, done, _, _ = env.step(action)

        print(f"Étape {episode + 1} - Action choisie : {action}, Récompense : {reward}, Nouvel état : {next_state}")

        total_reward += reward

    print(f"Récompense totale pour l'épisode {episode + 1}: {total_reward}\n")

env.close()


Étape 1 - Action choisie : 2, Récompense : -1, Nouvel état : 161
Étape 1 - Action choisie : 3, Récompense : -1, Nouvel état : 141
Étape 1 - Action choisie : 0, Récompense : -1, Nouvel état : 241
Étape 1 - Action choisie : 5, Récompense : -10, Nouvel état : 241
Étape 1 - Action choisie : 0, Récompense : -1, Nouvel état : 341
Étape 1 - Action choisie : 0, Récompense : -1, Nouvel état : 441
Étape 1 - Action choisie : 3, Récompense : -1, Nouvel état : 421
Étape 1 - Action choisie : 1, Récompense : -1, Nouvel état : 321
Étape 1 - Action choisie : 2, Récompense : -1, Nouvel état : 341
Étape 1 - Action choisie : 2, Récompense : -1, Nouvel état : 341
Étape 1 - Action choisie : 5, Récompense : -10, Nouvel état : 341
Étape 1 - Action choisie : 2, Récompense : -1, Nouvel état : 341
Étape 1 - Action choisie : 5, Récompense : -10, Nouvel état : 341
Étape 1 - Action choisie : 0, Récompense : -1, Nouvel état : 441
Étape 1 - Action choisie : 1, Récompense : -1, Nouvel état : 341
Étape 1 - Action chois

KeyboardInterrupt: 

### Exercice 3: Mise à jour de la politique avec PPO

In [8]:
class Policy:
    def __init__(self, action_size):
        self.action_size = action_size
        self.policy_table = np.ones(self.action_size) / self.action_size
    
    def get_action(self):
        action = np.random.choice(self.action_size, p=self.policy_table)
        return action
    
    def update(self, old_probs, new_probs, advantages, epsilon=0.2):
        ratio = new_probs / old_probs
        clipped_ratio = np.clip(ratio, 1 - epsilon, 1 + epsilon)
        objective = np.minimum(ratio * advantages, clipped_ratio * advantages)
        loss = -np.mean(objective)
        return loss

In [24]:
class ValueFunction:
    def __init__(self, state_size):
        self.value_table = np.zeros(state_size)
    
    def get_value(self, state):
        if not isinstance(state, int):
            state = state[0]
        return self.value_table[state]
    
    def update(self, states, discounted_rewards, alpha=0.01):
        for state, reward in zip(states, discounted_rewards):
            if not isinstance(state, int):
                state = state[0]
            self.value_table[state] += alpha * (reward - self.value_table[state])


In [25]:

env = gym.make("Taxi-v3")

# Hyperparamètres PPO
gamma = 0.99 
epsilon = 0.2
epochs = 1000
max_timesteps = 200


state_size = env.observation_space.n
action_size = env.action_space.n
policy = Policy(action_size)
value_function = ValueFunction(state_size)

def compute_discounted_rewards(rewards, gamma=0.99):
    discounted_rewards = []
    cumulative_reward = 0
    for reward in reversed(rewards):
        cumulative_reward = reward + gamma * cumulative_reward
        discounted_rewards.insert(0, cumulative_reward)
    return discounted_rewards

# Entraîner le modèle avec PPO
def train_ppo():
    for episode in range(epochs):
        state = env.reset()
        done = False
        episode_rewards = []
        old_probs = []
        new_probs = []
        values = []
        rewards = []
        states = []
        actions = []
        
        for t in range(max_timesteps):
            action = policy.get_action()
            old_prob = policy.policy_table[action]
            next_state, reward, done, _, _ = env.step(action)
            
            # Enregistrer la trajectoire
            states.append(state)
            actions.append(action)
            rewards.append(reward)
            old_probs.append(old_prob)
            
            # Calculer la valeur de l'état actuel
            value = value_function.get_value(state)
            values.append(value)
            
            state = next_state
            episode_rewards.append(reward)
            
            if done:
                break
        
        # Calcul des récompenses cumulées (Rt)
        discounted_rewards = compute_discounted_rewards(rewards, gamma)
        
        # Calcul des avantages (At = Rt - V(st))
        advantages = np.array(discounted_rewards) - np.array(values)
        
        # Mise à jour de la politique avec le clipping
        for action, old_prob, advantage in zip(actions, old_probs, advantages):
            new_prob = policy.policy_table[action]
            policy.update(old_prob, new_prob, advantage, epsilon)
        
        # Mise à jour de la fonction de valeur (V(s))
        value_function.update(states, discounted_rewards)

    
        print(f"Episode {episode+1} récompense totale : {sum(episode_rewards)}")

# le training
train_ppo()

env.close()


Episode 1 récompense totale : -866
Episode 2 récompense totale : -695
Episode 3 récompense totale : -839
Episode 4 récompense totale : -731
Episode 5 récompense totale : -722
Episode 6 récompense totale : -848
Episode 7 récompense totale : -704
Episode 8 récompense totale : -866
Episode 9 récompense totale : -830
Episode 10 récompense totale : -704
Episode 11 récompense totale : -758
Episode 12 récompense totale : -875
Episode 13 récompense totale : -821
Episode 14 récompense totale : -695
Episode 15 récompense totale : -758
Episode 16 récompense totale : -713
Episode 17 récompense totale : -911
Episode 18 récompense totale : -794
Episode 19 récompense totale : -695
Episode 20 récompense totale : -830
Episode 21 récompense totale : -812
Episode 22 récompense totale : -776
Episode 23 récompense totale : -893
Episode 24 récompense totale : -776
Episode 25 récompense totale : -722
Episode 26 récompense totale : -758
Episode 27 récompense totale : -695
Episode 28 récompense totale : -677
E

### Exercice 4: Evaluation de l'agent aprés entrainement

5